In [13]:
#imported libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#load the dataset
dataset1_path = '/content/drive/MyDrive/Colab Notebooks/players_preprocessed_new_data.csv'
dataset2_path = '/content/drive/MyDrive/Colab Notebooks/preprocessed_airlines_data (1).csv'

tweet_airline_sentiment = pd.read_csv(dataset2_path)
tweet_player_sentiment = pd.read_csv(dataset1_path)

In [4]:
# Change the column names
new_column_names = ['Unnamed: 0','text', 'airline_sentiment']
tweet_player_sentiment.columns = new_column_names

# Append dataset 10-14 to dataset 1-10
sentiment_data = tweet_airline_sentiment.append(tweet_player_sentiment)

# Optional: Reset the index of the merged dataset
sentiment_data = sentiment_data.reset_index(drop=True)

# Rename a single column
sentiment_data.rename(columns={'airline_sentiment': 'sentiment'}, inplace=True)

sentiment_data = sentiment_data.drop('Unnamed: 0', axis=1)

<ipython-input-4-3a331ad85b6f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_data = tweet_airline_sentiment.append(tweet_player_sentiment)


In [5]:
# Encode the sentiment labels
label_encoder = LabelEncoder()
sentiment_data['sentiment'] = label_encoder.fit_transform(sentiment_data['sentiment'])

In [14]:
# train, val, test split

x_train, xtest, y_train, ytest = train_test_split(sentiment_data.text.values,sentiment_data.sentiment,stratify=sentiment_data.sentiment.values, test_size=0.3,random_state=25)
y_train = to_categorical(y_train)
x_val = xtest[0:10000]
y_val = to_categorical(ytest[0:10000])
x_test = xtest[10000:]
y_test = ytest[10000:]

# Convert non-string values to strings
sentiment_data.text = sentiment_data.text.astype(str)
x_train = x_train.astype(str)
x_val = x_val.astype(str)
x_test = x_test.astype(str)

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentiment_data.text.values)


X_train = tokenizer.texts_to_sequences(x_train)
X_val = tokenizer.texts_to_sequences(x_val)
X_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100
#padding
X_train = pad_sequences(X_train, padding='pre', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='pre', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='pre', maxlen=maxlen)

word_index = tokenizer.word_index

In [16]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
# Load the trained models
simple_rnn_model = load_model('/content/drive/MyDrive/Colab Notebooks/simplernn-model.h5')
lstm_model = load_model('/content/drive/MyDrive/Colab Notebooks/rnn-lstmmodel.h5')
cnn_model = load_model('/content/drive/MyDrive/Colab Notebooks/cnn-model.h5')

# Make predictions using each model
simple_rnn_preds = simple_rnn_model.predict(X_test)
lstm_preds = lstm_model.predict(X_test)
cnn_preds = cnn_model.predict(X_test)

# Combine predictions using voting mechanism
ensemble_preds = []
for i in range(len(X_test)):
    # Perform majority voting
    simple_rnn_pred = np.argmax(simple_rnn_preds[i])
    lstm_pred = np.argmax(lstm_preds[i])
    cnn_pred = np.argmax(cnn_preds[i])
    
    final_pred = np.argmax(np.bincount([simple_rnn_pred, lstm_pred, cnn_pred]))
    ensemble_preds.append(final_pred)

# Evaluate the ensemble predictions
accuracy = accuracy_score(y_test, ensemble_preds)
print("Ensemble Accuracy:", accuracy)


2819/2819 [==============================] - 58s 21ms/step
Ensemble Accuracy: 0.8423912440812162


Accuracy 84.5 

In [18]:
#map the sentiment 
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [20]:
input_text = "I was extremely disappointed with the service I received at the restaurant. The staff was rude and inattentive, and the food took forever to arrive. Not only was the quality of the food below average, but it was also overpriced. The overall experience left a sour taste in my mouth, and I would not recommend this place to anyone. It's a shame because I had high expectations, but they were completely let down. I hope they can improve their service and quality in the future."  # Replace with your input text

input_sequence = tokenizer.texts_to_sequences([input_text])
input_sequence = pad_sequences(input_sequence, padding='pre', maxlen=maxlen)

# Make predictions using each model
simple_rnn_pred = simple_rnn_model.predict(input_sequence)
lstm_pred = lstm_model.predict(input_sequence)
cnn_pred = cnn_model.predict(input_sequence)

# Combine predictions using voting mechanism
ensemble_pred = np.argmax(np.bincount([np.argmax(simple_rnn_pred), np.argmax(lstm_pred), np.argmax(cnn_pred)]))

# Map the sentiment label to its corresponding class
sentiment_label = label_mapping[ensemble_pred]

print("Input Text:", input_text)
print("Predicted Sentiment:", sentiment_label)


1/1 [==============================] - 0s 34ms/step
Input Text: I was extremely disappointed with the service I received at the restaurant. The staff was rude and inattentive, and the food took forever to arrive. Not only was the quality of the food below average, but it was also overpriced. The overall experience left a sour taste in my mouth, and I would not recommend this place to anyone. It's a shame because I had high expectations, but they were completely let down. I hope they can improve their service and quality in the future.
Predicted Sentiment: Negative
